# Time in ROI demo notebook

In [2]:
import sys
import numpy as np
dlc_dir = '/Users/annateruel/phd_code/'
if dlc_dir not in sys.path:
    sys.path.append(dlc_dir)

import os
import pandas as pd
import dlc.analysis.time_roi as time_roi
from dlc.analysis.time_roi import ROIDrawer, DynamicROI
from dlc.data import Concatenate, Centroid, Interpolation
from dlc.plotting import TrackingPlot, InterpolationPlot
from dlc.load_data import DataLoader

### Concatenate your videos
Concatenate your data from h5 and video files (for data recorded with Miniscope Software)

In [ ]:
dir = '/Users/annateruel/Desktop/poster_sfn/calcium/dlc'
concatenator = Concatenate(dir)
h5_concat= concatenator.concatenate_h5()
video_concat = concatenator.concatenate_videos('.avi')

In [3]:
root_dir = '/Users/annateruel/Desktop/poster_sfn/calcium/dlc'

for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        for condition in ['control', 'anosmic']:
            condition_dir = os.path.join(subdir_path, condition)
            
            if os.path.exists(condition_dir):
                concatenator = Concatenate(condition_dir)
                h5_concat = concatenator.concatenate_h5()
                print(f"h5 concatenation result: {h5_concat}")
                video_concat = concatenator.concatenate_videos('.avi')
                print(f"Video concatenation result: {video_concat}")

### ROI Drawer tool

In [ ]:
video = '/Users/annateruel/Desktop/AD23-229-hab2-2.mp4'
savedir = os.path.dirname(video)
roi_drawer = ROIDrawer(video_path=video,save_dir=savedir, num_rois=2)
r = roi_drawer.draw_rois()

### Interpolate bodyparts

Before calculating time in a region of interest, we will begin by interpolating all bodyparts to make sure that the tracking is fine-tuned

In [ ]:
input_data = '/Users/annateruel/Desktop/h5s/raw/' #raw folders includes files without the inteprolation
bodyparts = ['nose', 
             'rightear', 
             'leftear', 
             'head', 
             'spine1', 
             'spine2', 
             'basetail', 
             'lc', 
             'lr', 
             'rc', 
             'rr'] 
output_dir = '/Users/annateruel/Desktop/h5s/'
interpolator = Interpolation(threshold=0.995, interpolation_method='linear')
interpolated_data = interpolator.interpolate_data(input_data, bodyparts, output_dir=output_dir)

### Plot interpolation

We're going to plot comparison between before-after interpolation for each body part in our dataframe. 

In [ ]:
interpolator = Interpolation(threshold=0.995, interpolation_method='linear')
plotter = InterpolationPlot(interpolator, style="light", colormap="RdYlGn", linewidth=0.5, alpha=0.90)

input_dir = '/Users/annateruel/Desktop/h5s/raw/'
output_dir = '/Users/annateruel/Desktop/h5s/plots/'
plotter.plot_directory(input_dir, 
                       ['nose', 
                        'rightear', 
                        'leftear', 
                        'head', 
                        'spine1', 
                        'spine2', 
                        'basetail', 
                        'lc', 
                        'lr', 
                        'rc', 
                        'rr'],
                        'Comparison of Interpolated Data', 
                        output_dir)

### Calculate the centroid of your body parts

Before calculating time ina region of interest, we will begin by getting the centroid of x bodyparts

In [3]:
dir = '/Users/annateruel/Desktop/h5s/raw'
bodyparts = ["nose", "rightear", "leftear", "head", "spine1", "spine2"] 

centroid = Centroid(minutes = 5)
result = centroid.get_centroid(dir, bodyparts, save = True, suffix='filtered.h5')

for key, df in result.items():
        df.to_hdf(os.path.join(dir, f"{key}_centroid.h5"), key='df', mode='w')

### Time in ROI

In [ ]:
video_dir = '/Users/annateruel/Desktop/AD23-120/DLC'
scorer = 'DLC_resnet50_sr-ca2+imgNov17shuffle1_250000' #should take it from DLC multiindex dataframe
bodypart = 'centroid'

In [ ]:
roi_ellipse = {}
for file in os.listdir(video_dir):
    if file.endswith('roi.h5') and not file.startswith('._'):
        roi_file_path = os.path.join(video_dir, file)
        print(f"Processing ROI file: {file}")

        try:
            roi_data = pd.read_hdf(roi_file_path)
            roi_groups = roi_data.groupby(['index', 'shape-type'])
            ellipse, _ = time_roi.EllipseROI.extract_ellipses(roi_groups)
            key = file.replace('_roi.h5', '')
            roi_ellipse[key] = ellipse
            print(f"Extracted ellipses from {file}: {len(ellipse)}")
        except Exception as e:
            print(f"Error processing {file}: {e}")

In [ ]:
frames_with_zero_rois = []
for key, ellipses in roi_ellipse.items():
    if len(ellipses) == 0:
        frames_with_zero_rois.append(key)
print("Frames with 0 ROIs:")

for frame in frames_with_zero_rois:
    print(frame)


In case you want to check the coords for each roi, you can use following cell to print it

In [ ]:
print("\nContents of roi_ellipse dictionary:")
for key, ellipses in roi_ellipse.items():
    print(f"Key: {key}, Number of ROIs: {len(ellipses)}")
    for i, ellipse in enumerate(ellipses):
        print(f"  ROI {i}: Center={ellipse.center}, Radii={ellipse.rad}")
        

In [ ]:
rois = roi_ellipse['AD23-120-s2_concatenated']
time_in_roi = time_roi.TimeinRoi(fps=60)
file_path = '/Users/annateruel/Desktop/AD23-120/DLC/AD23-120-s2_centroid.h5'
for roi in rois:
    time_in_roi.add_roi(roi)

tracking_data = time_in_roi.extract_tracking_data(file_path, scorer, bodypart)
time_in_rois = time_in_roi.time_in_rois(tracking_data)
print(time_in_rois)

Because each video might have slightly different fps, let's get a specific value for each video


In [ ]:
time_in_roi = time_roi.TimeinRoi(fps=60)
results_df = time_in_roi.time_in_rois_dir(directory=video_dir, 
                                          rois=roi_ellipse, 
                                          scorer=scorer, 
                                          body_part=bodypart, 
                                          file_endswith='_centroid.h5', 
                                          filename_replace = 'filtered.h5_centroid.h5')
results_df.to_hdf('/Users/annateruel/Desktop/results.h5', key='df', mode='w')
print(results_df)

### Time in dynamic ROI

In [8]:
import os
import pandas as pd

h5_directory = '/Users/annateruel/Desktop/h5s/centroid/'  
video_directory = '/Users/annateruel/Desktop/h5s/centroid/'

total_times = {}
for h5_filename in os.listdir(h5_directory):
    if h5_filename.endswith("centroid.h5"):
        h5_filepath = os.path.join(h5_directory, h5_filename)
        roi = DynamicROI(c='lc', r='lr', video_dir=video_directory)
        video_filepath = roi.map_videofile(h5_filename)
        if video_filepath is None:
            print(f"No video file found for {h5_filename}")
            continue
        
        df = pd.read_hdf(h5_filepath)
        total_time, frames_in_roi = roi.dynamic_time_in_roi(df, video_filepath, minutes=5)
        total_times[h5_filename] = (total_time, frames_in_roi)

results_df = pd.DataFrame(
    [(filename, data[0], data[1]) for filename, data in total_times.items()],
    columns=['filename', 'total_time_in_roi', 'frames_in_roi']
)

output_filepath = '/Users/annateruel/Desktop/results_left5.csv'
results_df.to_csv(output_filepath, index=False)
print(f"Results saved to {output_filepath}")

FPS for /Users/annateruel/Desktop/h5s/centroid/AD23-239-hab1.mp4: 30.0
Calculating dynamic time in ROI for the first 5 minutes (9000 frames).
FPS for /Users/annateruel/Desktop/h5s/centroid/AD22-024-TRN1_bw.mp4: 23.976023976023978
Calculating dynamic time in ROI for the first 5 minutes (7192 frames).
FPS for /Users/annateruel/Desktop/h5s/centroid/AD22-155-hab1_bw.mp4: 14.985014985014985
Calculating dynamic time in ROI for the first 5 minutes (4495 frames).
FPS for /Users/annateruel/Desktop/h5s/centroid/AD23-239-trn2.mp4: 30.0
Calculating dynamic time in ROI for the first 5 minutes (9000 frames).
FPS for /Users/annateruel/Desktop/h5s/centroid/AD22-152-trn2_bw.mp4: 14.985014985014985
Calculating dynamic time in ROI for the first 5 minutes (4495 frames).
FPS for /Users/annateruel/Desktop/h5s/centroid/AD22-083-HAB2_bw.mp4: 23.976023976023978
Calculating dynamic time in ROI for the first 5 minutes (7192 frames).
FPS for /Users/annateruel/Desktop/h5s/centroid/AD22-083-TEST_bw.mp4: 23.97602397

### Heatmaps or Density plots

In [ ]:
dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/control/'
bodyparts = ['nose', 'rightear', 'leftear', 'head']

In [ ]:
tracking_plot = TrackingPlot(style="light")

tracking_plot.plot_directory(
    dir_path=dir,
    bodyparts=bodyparts,
    title='Density plot of head position',
    plot_type='density',
    save=True,
    save_directory=dir,
    cmap='summer',
    width=11.34,
    height=3.35)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os

# Define the directory containing the .h5 files
input_directory = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-112/anosmic/concatenated_files'   # Replace with your actual input directory path

# Get all .h5 files ending with 'filtered.h5' in the directory
file_list = glob.glob(f'{input_directory}/*anosmic_concatenated.h5')

# Loop through each file and generate a KDE plot
for file in file_list:
    # Extract the directory and filename without the extension
    file_dir = os.path.dirname(file)
    file_name = os.path.splitext(os.path.basename(file))[0]

    # Create a density directory inside the file's directory
    density_directory = os.path.join(file_dir, 'density')
    os.makedirs(density_directory, exist_ok=True)

    # Load the data
    data = pd.read_hdf(file)
    
    # Extract the coordinates for the "head" body part
    head_x = data.loc[:, (slice(None), "spine1", "x")].values.flatten()
    head_y = data.loc[:, (slice(None), "spine1", "y")].values.flatten()

    # Plot the density plot
    plt.figure(figsize=(10, 6))
    ax = sns.kdeplot(x=head_x, y=head_y, fill=True, cmap='summer', levels=10, alpha=0.9)

    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title(f'Density Plot for Head Coordinates - {file_name}')

    # Save the plot as an SVG file
    output_file_path = os.path.join(density_directory, f'{file_name}.svg')
    plt.savefig(output_file_path, format='svg')

    # Close the plot to free up memory
    plt.close()

print(f'Plots saved to "density" directories within {input_directory}')



### Time in ROI bins 
